In [1]:
import cv2
import numpy as np
import os
import shutil
import pandas as pd

def find_intersections_and_connect(image_path, y_level, distance_threshold=2):
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"未找到或无法读取图像: {image_path}")
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        intersections = []

        for contour in contours:
            for i in range(len(contour) - 1):
                pt1 = tuple(contour[i][0])
                pt2 = tuple(contour[i + 1][0])

                if (pt1[1] - y_level) * (pt2[1] - y_level) <= 0:
                    if pt1[1] != pt2[1]:
                        x_intersect = int(pt1[0] + (pt2[0] - pt1[0]) * (y_level - pt1[1]) / (pt2[1] - pt1[1]))
                        intersection = (x_intersect, y_level)
                        
                        # 检查该交点与现有交点的距离是否小于阈值
                        if not intersections or all(np.linalg.norm(np.array(intersection) - np.array(existing_point)) > distance_threshold for existing_point in intersections):
                            intersections.append(intersection)

        if intersections:
            intersections = sorted(intersections)
            longest_segment = None
            max_length = 0
            for i in range(len(intersections) - 1):
                length = np.linalg.norm(np.array(intersections[i]) - np.array(intersections[i + 1]))
                if length > max_length:
                    max_length = length
                    longest_segment = (intersections[i], intersections[i + 1])

            if longest_segment:
                midpoint = ((longest_segment[0][0] + longest_segment[1][0]) // 2,
                             (longest_segment[0][1] + longest_segment[1][1]) // 2)
                return midpoint[0]  # 返回中点的横坐标
        else:
            img_width = img.shape[1]  # 获取图像的宽度
            return img_width // 2  # 返回图像中点的 x 值

    except Exception as e:
        print(f"处理图像 {image_path} 时出错: {e}")

    # 如果发生异常或其他未处理情况，返回图像的中点横坐标
    return 0

def process_images_in_folder(folder_path, y_levels):
    # 创建目标文件夹，仅保留left和right文件夹
    left_folder = os.path.join(folder_path, 'left')
    right_folder = os.path.join(folder_path, 'right')

    for folder in [left_folder, right_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    all_midpoints = []  # 用来存储所有图像的横坐标均值

    # 遍历文件夹中的图像文件
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # 支持的图像格式
            image_path = os.path.join(folder_path, filename)
            midpoints_x = []

            # 对每个 y_level 计算交点的横坐标
            for y_level in y_levels:
                midpoint_x = find_intersections_and_connect(image_path, y_level)
                
                # 如果横坐标为0，将其视作图像中点的横坐标
                if midpoint_x == 0:
                    img = cv2.imread(image_path)
                    img_width = img.shape[1]
                    midpoint_x = img_width // 2  # 将0视作图像的中点横坐标

                midpoints_x.append(midpoint_x)

            # 计算有效横坐标的均值
            if midpoints_x:
                mean_x = np.mean(midpoints_x)
            else:
                mean_x = 0  # 如果没有有效横坐标，均值为0

            # 获取图像的宽度（中点横坐标）
            img = cv2.imread(image_path)
            img_width = img.shape[1]
            image_midpoint_x = img_width // 2

            # 根据均值横坐标进行分类
            if mean_x < img_width / 2:  # 将左侧图像分类为left
                shutil.copy(image_path, os.path.join(left_folder, filename))  # 偏左
            else:  # 右侧图像分类为right
                shutil.copy(image_path, os.path.join(right_folder, filename))  # 偏右

            all_midpoints.append({'Image': filename, 'Mean_X': mean_x, 'Image_Midpoint_X': image_midpoint_x})

    # 保存均值数据到 CSV 文件
    df = pd.DataFrame(all_midpoints)
    df.to_csv('midpoint_means.csv', index=False)
    print("图像处理完毕，均值数据已保存到 'midpoint_means.csv'。")



In [5]:
# 使用示例
folder_path = r"C:\Users\yil\Desktop\ipy project\bot-project\transfer_picture\black-white"  # 图像文件夹路径
y_levels = [0,10,20,30,40,50,60,70,80,90,100]  # 你想分析的不同水平线的纵坐标
process_images_in_folder(folder_path, y_levels)

print("已处理文件夹中的所有图像，中点的横坐标已保存到 midpoints.csv")

图像处理完毕，均值数据已保存到 'midpoint_means.csv'。
已处理文件夹中的所有图像，中点的横坐标已保存到 midpoints.csv
